# Elfskot PyAPI

In [1]:
# Required packages:
# pip3 install requests pandas
import requests
import json
from enum import Enum
import pandas as pd

def head(s): return s[0] if len(s) > 0 else []
def tail(s): return s[1:]
def reverse(s): return s[::-1]
def last(s): return s[-1:]
def init(s): return s[0:len(s)-1]
def take(s, n): return s[:n]
def drop(s, n): return s[n:]
def product(s): return reduce(lambda x, y: x * y, s)

class TextType(Enum):
    Description = 0
    ExtendedDescription = 1
    MoreInfo = 2

class ElfskotApi():
    
    base_address = 'https://api.elfskot.cloud/api/2/'
    
    def __init__(self, application_id, secret):
        self.application_id = application_id
        self.secret = secret
        self.get_token()
    
    def get_token(self):
        payload = { 'clientId': self.application_id, 'secret': self.secret}
        result = requests.post(self.get_url('auth/elfskotconnectlogin'), json=payload)
        self.check_result(result)
        self.token = json.loads(result.text)['accessToken']
        
    def check_result(self, result):
        if result.status_code != 200:
            raise ValueError('Error: API returned status code {}'.format(result.status_code))
        return True
        
    def get_auth_header(self): return {'Authorization': 'bearer {}'.format(self.token)}
    def get_url(self, endpoint): return self.base_address + endpoint
    
    def http_return_if_valid(self, result):
        self.check_result(result)
        return json.loads(result.text)
    
    def http_get(self, endpoint):
        return self.http_return_if_valid(
            requests.get(self.get_url(endpoint), headers=self.get_auth_header())
        )
    
    def http_post(self, endpoint, o):
        return self.http_return_if_valid(
            requests.post(self.get_url(endpoint), json=o, headers=self.get_auth_header())
        )
    
    def http_put(self, endpoint, o):
        return self.http_return_if_valid(
            requests.put(self.get_url(endpoint), json=o, headers=self.get_auth_header())
        )
    
    def http_delete(self, endpoint, key):
        result = requests.delete(self.get_url(endpoint) + '/{}'.format(key), headers=self.get_auth_header())
        if result.status_code != 200: 
            raise ValueError('Error: API returned status code {}'.format(result.status_code))
            
    def all(self, endpoint): return self.http_get(endpoint)
    def get(self, endpoint, k): return head(self.http_get('{}?id={}'.format(endpoint, k)))
    def find(self, endpoint, p, v): return self.http_get('{}?{}={}'.format(endpoint, p, v))
    def first(self, endpoint, p, v): return head(self.find(endpoint, p, v))
    def new(self, endpoint, o): return self.http_post(endpoint, o)
    def update(self, endpoint, o): return self.http_put(endpoint, o)
    def delete(self, endpoint, k): self.http_delete(endpoint, k)  
    def help(self, endpoint): print('Model for {}:\r\n{}'
                                    .format(endpoint, list(self.first(endpoint, 'id', '').keys())))

In [284]:
db = ElfskotApi('69b1132c-df11-49d0-baf2-2d8f6ddfc4f1', 'lkmx41yu')

category = head(db.all('categories'))
uom = db.first('uom', 'description', 'Milli')

feature = {'name': 'Test feature', 
           'articleCode': 'ART-123', 
           'categoryId': category['id'], 
           'unitOfMeasurementId': uom['id'],
           'texts': [
               {'languageIso': 'en', 'type': TextType.Description.value, 'value': 'Example description'},
               {'languageIso': 'en', 'type': TextType.ExtendedDescription.value, 'value': 'Example description'},
               {'languageIso': 'en', 'type': TextType.MoreInfo.value, 'value': 'Example description'}
           ]}

feature = db.new('features', feature)
db.delete('features', feature['id'])

In [285]:
len(db.all('features'))

245

In [263]:
db.help('featuremodels')

Model for featuremodels:
['rootFeatureId', 'status', 'revisionOf', 'rootFeature', 'order', 'displayPrices', 'autodeskUrn', 'steps', 'nodes', 'relationships', 'organizationSellsFeatureModels', 'dynamicGroups', 'id', 'organizationId', 'organizationName', 'creatorId', 'reference', 'synced', 'createdDate', 'updatedDate', 'customField1', 'customField2', 'customField3', 'customField4', 'customField5']


In [252]:
db.help('quotations')

Model for quotations:
['wholeSaleDiscountRows', 'quotationNumber', 'versionNumber', 'revisionOfId', 'status', 'isExpired', 'statusDescription', 'remarks', 'subject', 'expiresDate', 'deliverydate', 'debtorId', 'debtor', 'sellerLogo', 'sellerId', 'seller', 'shipToId', 'shipTo', 'propertyIds', 'margin', 'leaseTermType', 'leaseTermAmount', 'lines', 'materialList', 'currencyIso', 'languageIso', 'currency', 'exchangeRate', 'quotationTemplateId', 'quotationTemplate', 'files', 'totalDiscountAmt', 'totalDiscountPct', 'subTotalExclPrice', 'subTotalInclPrice', 'subTotalExclPriceLabel', 'subTotalInclPriceLabel', 'totalDiscountAmntLabel', 'totalExclPrice', 'totalInclPrice', 'totalPriceExclVATLabel', 'totalPriceInclVAT', 'totalPriceInclVATLabel', 'subTotalMargin', 'totalMargin', 'totalMarginLabel', 'totalMarginDiscount', 'totalMarginDiscountLabel', 'subTotalPurchasePrice', 'subTotalPurchasePriceLabel', 'subTotalMarginLabel', 'totalPurchasePriceLabel', 'wholeSaleDiscountAmtLabel', 'wholeSaleDiscountM

In [271]:
# Fetch all root features
for key in map(lambda x: x['rootFeatureId'], db.all('featuremodels')):
    feature = db.get('features', key)
    print(feature['name'])

S6000
X8000
Plateauaanhanger
TEST relaties
Betonput
Test Wessel
Formule cycle
b_min


In [282]:
df = pd.DataFrame()
N = 5
names = ['b{}'.format(x) for x in range(N)]
codes = ['article-code'] * N
df['name'] = names
df['articleCode'] = codes
print(df)
for index, row in df.iterrows():
    feature = {'name': row['name'], 'articleCode': row['articleCode']}
    feature = db.new('features', feature)
    print('Created feature {} with id {}.'.format(feature['name'], feature['id']))
    db.delete('features', feature['id'])

  name   articleCode
0   b0  article-code
1   b1  article-code
2   b2  article-code
3   b3  article-code
4   b4  article-code
Created feature b0 with id f598729a-3f0b-4ca7-82fa-08d649534e0a.
Created feature b1 with id 776a36fa-bbf6-4856-82fb-08d649534e0a.
Created feature b2 with id 1dfbf79f-95b5-45d0-82fc-08d649534e0a.
Created feature b3 with id 77ad541a-fa9b-49ce-82fd-08d649534e0a.
Created feature b4 with id ad2303ff-4408-4690-82fe-08d649534e0a.


In [2]:
db = ElfskotApi('71ac0e2b-b8a6-4d3f-84bb-ace0d0c43148', 'cwnwkiff')

In [3]:
db.all('features')

[{'articleCode': 'X_108',
  'cardImageUrl': 'https://elfskotcdn.blob.core.windows.net/a4ece79f-f6f0-4c80-9cba-08d4ad0d854f/Elfvo.jpg',
  'category': None,
  'categoryId': '4fde3e79-c90e-4844-3d1b-08d50589c407',
  'createdDate': '2018-01-25T16:20:08.4867666+01:00',
  'creatorId': '00000000-0000-0000-0000-000000000000',
  'customField1': None,
  'customField2': None,
  'customField3': None,
  'customField4': None,
  'customField5': None,
  'hiddenThreeDModelItems': [],
  'id': '01574434-b7f4-42d4-b518-08d4ad0fbdd7',
  'marginPct': 0.0,
  'maxValue': 0.0,
  'minValue': 0.0,
  'name': 'S6000',
  'organizationId': None,
  'organizationName': None,
  'organizationSellsFeature': None,
  'packingUnit': 0.0,
  'properties': None,
  'reference': '01d0e364-5ae1-4506-99a3-935d3634efa4',
  'salesPrice': 42750.0,
  'salesPriceLabel': '€ 42.750,00',
  'stepValue': 0.0,
  'subcategoryIds': ['5cec6233-c88a-4cca-dbc2-08d5cfa773a6',
   '5da9354e-60f3-491d-4291-08d5fddd4fac',
   '831a6ef6-d3d1-48e8-4292-0

In [4]:
db.first('features', 'description', 'S6000')

{'articleCode': 'X_108',
 'cardImageUrl': 'https://elfskotcdn.blob.core.windows.net/a4ece79f-f6f0-4c80-9cba-08d4ad0d854f/Elfvo.jpg',
 'category': None,
 'categoryId': '4fde3e79-c90e-4844-3d1b-08d50589c407',
 'createdDate': '2018-01-25T16:20:08.4867666+01:00',
 'creatorId': '00000000-0000-0000-0000-000000000000',
 'customField1': None,
 'customField2': None,
 'customField3': None,
 'customField4': None,
 'customField5': None,
 'hiddenThreeDModelItems': [],
 'id': '01574434-b7f4-42d4-b518-08d4ad0fbdd7',
 'marginPct': 0.0,
 'maxValue': 0.0,
 'minValue': 0.0,
 'name': 'S6000',
 'organizationId': None,
 'organizationName': None,
 'organizationSellsFeature': None,
 'packingUnit': 0.0,
 'properties': None,
 'reference': '01d0e364-5ae1-4506-99a3-935d3634efa4',
 'salesPrice': 42750.0,
 'salesPriceLabel': '€ 42.750,00',
 'stepValue': 0.0,
 'subcategoryIds': ['5cec6233-c88a-4cca-dbc2-08d5cfa773a6',
  '5da9354e-60f3-491d-4291-08d5fddd4fac',
  '831a6ef6-d3d1-48e8-4292-08d5fddd4fac'],
 'synced': Fal

In [5]:
db.first('uom', 'code', 'mm')

{'code': 'mm',
 'createdDate': '2018-10-30T12:43:54.2744502+00:00',
 'creatorId': '7c10626d-6d57-403c-ec51-08d56407f341',
 'customField1': None,
 'customField2': None,
 'customField3': None,
 'customField4': None,
 'customField5': None,
 'description': 'Millimeter',
 'id': '6ba61d80-cf85-4607-570e-08d63e655633',
 'organizationId': None,
 'organizationName': None,
 'reference': None,
 'synced': False,
 'updatedDate': '2018-11-09T14:29:39.8466742+00:00'}

In [7]:
db.help('featuremodels')

Model for featuremodels:
['rootFeatureId', 'status', 'revisionOf', 'rootFeature', 'order', 'displayPrices', 'autodeskUrn', 'steps', 'nodes', 'relationships', 'organizationSellsFeatureModels', 'dynamicGroups', 'id', 'organizationId', 'organizationName', 'creatorId', 'reference', 'synced', 'createdDate', 'updatedDate', 'customField1', 'customField2', 'customField3', 'customField4', 'customField5']


In [10]:
for key in map(lambda x: x['rootFeatureId'], db.all('featuremodels')):
    feature = db.get('features', key)
    print(feature['name'])

S6000
X8000
Plateauaanhanger
TEST relaties
Betonput
Test Wessel
Formule cycle
b_min


In [11]:
df = pd.DataFrame()
var = [1,2,3,4,5]
df['var'] = var
df

,var
0,1
1,2
2,3
3,4
4,5


In [15]:
{'yo': 'test'}['yo']

'test'